<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/finetuning_jeremy_su1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Settings

# 2. Model Load

# 3. Load Dataset

# 4. Model Layer

# 5. Fine-tuning

# 6. Evaluation